In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

# Percorso della cartella dove sono memorizzati i file zip nel tuo Google Drive
zip_folder_path = '/content/drive/MyDrive/uni/proj_social_deep_learning/dataset/'

# Percorso dove vuoi estrarre i file
extraction_path = '/content/dataset/'

for i in range(1, 13):
    zip_file_path = os.path.join(zip_folder_path, f"{str(i).zfill(2)}.zip")
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extraction_path)


In [ ]:
import cv2

def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Converti in RGB
    image = image / 255.0  # Normalizza i valori dei pixel
    image = image.astype(np.float32)
    return image

def crop_face(image, landmarks, expansion_factor=0.2):
    x_coordinates = [int(landmark[0]) for landmark in landmarks]
    y_coordinates = [int(landmark[1]) for landmark in landmarks]

    x_min, x_max = min(x_coordinates), max(x_coordinates)
    y_min, y_max = min(y_coordinates), max(y_coordinates)

    # Calcolo dell'intervallo
    x_range = x_max - x_min
    y_range = y_max - y_min

    # Aggiustamento per l'expansion factor
    x_min = max(x_min - int(x_range * expansion_factor), 0)
    x_max = min(x_max + int(x_range * expansion_factor), image.shape[1])
    y_min = max(y_min - int(y_range * expansion_factor), 0)
    y_max = min(y_max + int(y_range * expansion_factor), image.shape[0])

    # Verifica che le coordinate di ritaglio siano valide
    if x_min >= x_max or y_min >= y_max:
        print(f"Errore di calcolo del ritaglio: x_min={x_min}, x_max={x_max}, y_min={y_min}, y_max={y_max}")
        return None

    cropped_face = image[y_min:y_max, x_min:x_max]
    return cropped_face


In [ ]:
import os
import cv2
import numpy as np
import json

def process_video_folder(folder_path, folder_name, save_path):
    images, labels = [], []

    json_path = os.path.join(folder_path, f'{folder_name}.json')
    with open(json_path, 'r') as file:
        json_data = json.load(file)

    for frame_id, frame_info in json_data['frames'].items():
        image_path = os.path.join(folder_path, f"{frame_id}.png")

        image = load_and_preprocess_image(image_path)

        landmarks = frame_info['landmarks']
        cropped_face = crop_face(image, landmarks)
        cropped_face = (cropped_face * 255).astype(np.uint8)
        #resize img to 224,224
        # Salvataggio dell'immagine ritagliata
        resized_cropped_face = cv2.resize(cropped_face, (224,224))  # Ridimensiona l'immagine
        save_folder = os.path.join(save_path, folder_name)
        os.makedirs(save_folder, exist_ok=True)  # Crea la cartella se non esiste
        save_image_path = os.path.join(save_folder, f"{frame_id}.png")
        cv2.imwrite(save_image_path, cv2.cvtColor(resized_cropped_face, cv2.COLOR_RGB2BGR))

        images.append(resized_cropped_face)
        labels.append((frame_info['valence'], frame_info['arousal']))

    return images, labels

In [ ]:
import gc
# Pulizia della memoria
gc.collect()

0

In [ ]:
import sys


# Percorso dove salverai i dati ridimensionati
save_path = 'dataset_resized'

# Percorso dove hai estratto i dati
extraction_path = 'dataset'

processed_folders = 0
total_folders = len(os.listdir(extraction_path))

X, Y = [], []

for folder_name in os.listdir(extraction_path):
    folder_path = os.path.join(extraction_path, folder_name)
    if os.path.isdir(folder_path):
        images, labels = process_video_folder(folder_path, folder_name, save_path)
        X.extend(images)
        Y.extend(labels)
        processed_folders += 1
        sys.stdout.write(f"\rCartelle processate: {processed_folders}/{total_folders}")
        sys.stdout.flush()

print("\nElaborazione completata.")

gc.collect()

Cartelle processate: 600/601
Elaborazione completata.


0

# Load AU MODEL

In [ ]:
import cv2
import numpy as np
import dlib
import pandas as pd

def detect_landmarks(image_path):
    # Load the pre-trained face detector model
    detector_face = dlib.get_frontal_face_detector()
    predictor_path = "models/shape_predictor_68_face_landmarks.dat"
    predictor = dlib.shape_predictor(predictor_path)
    # Load an image
    image = cv2.imread(image_path)

    # Convert the image to grayscale (required by Dlib)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = detector_face(gray)

    landmarks = predictor(gray, faces[0])
    landmarks_coordinates = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(68)])

    return landmarks_coordinates


def landmarks_combination_df(landmarks):
    columns_names = [f'Landmark_x_{i}' for i in range(68)]
    columns_names1 = [f'Landmark_y_{i}' for i in range(68)]

    all_columns_names = columns_names + columns_names1

    vec_comb_lin_x = []
    vec_comb_lin_y = []

    # Dividi l'array in coordinate x e y
    land_x = landmarks[:, 0]  # Estrae tutte le colonne 0 (coordinate x)
    land_y = landmarks[:, 1]  # Estrae tutte le colonne 1 (coordinate y)

    for i in range(68):
        sum_x = 0
        sum_y = 0

        for j in range(68):
            sum_x = sum_x + land_x[i] * (1 / land_x[j])
            sum_y = sum_y + land_y[i] * (1 / land_y[j])

        vec_comb_lin_x.append(sum_x)
        vec_comb_lin_y.append(sum_y)

    all_landmark = vec_comb_lin_x + vec_comb_lin_y

    return pd.DataFrame([all_landmark], columns=all_columns_names)

In [ ]:
import os
import pandas as pd
import json
from tensorflow.keras.models import load_model

def process_all_images(folder_path, start_folder=0, end_folder=None):
    """
    Processa tutte le immagini in un dato percorso per estrarre le Action Units (AU).

    :param folder_path: Percorso della cartella contenente le immagini.
    :param start_folder: Indice della sotto-cartella da cui iniziare l'elaborazione.
    :param end_folder: Indice della sotto-cartella in cui fermare l'elaborazione (esclusiva).
    :return: DataFrame con i valori delle AU per ogni immagine processata.
    """
    aus_results = []
    au_pred_model = load_model('models/au_pred_model1.h5')
    sorted_folders = sorted(os.listdir(folder_path))

    # Se end_folder non è specificato, processa fino all'ultima cartella
    if end_folder is None:
        end_folder = len(sorted_folders)

    # Processa solo le cartelle nell'intervallo specificato
    for folder_index, folder_name in enumerate(sorted_folders[start_folder:end_folder], start_folder):
        subfolder_path = os.path.join(folder_path, folder_name)
        print(f"Elaborazione della sotto-cartella: {subfolder_path}")

        if not os.path.isdir(subfolder_path):
            print(f"Non è una cartella: {subfolder_path}")
            continue

        for image_name in os.listdir(subfolder_path):
            image_path = os.path.join(subfolder_path, image_name)
            if not image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue  # Process only image files

            try:
                # Esegui la detection delle AU
                landmarks = detect_landmarks(image_path)
                if len(landmarks) > 0:
                    landmarks_df = landmarks_combination_df(landmarks)
                    prediction = au_pred_model.predict(landmarks_df,verbose=0)
                    au_values = prediction[0]  # Presumiamo che questo sia un array con le previsioni delle AU

                # Crea un dizionario dalle previsioni delle AU utilizzando indici numerici come chiavi
                au_dict = {'AU' + str(index + 1): value for index, value in enumerate(au_values)}

                # Crea il dizionario finale aggiungendo il percorso dell'immagine
                au_values_with_path = {'image_path': image_path, **au_dict}
                aus_results.append(au_values_with_path)
            except Exception as e:
                print(f"Errore durante l'elaborazione di {image_path}: {e}")

    return pd.DataFrame(aus_results)




In [ ]:
# Percorso della cartella principale del dataset
dataset_path = "/content/dataset_resized/"

# Processa le immagini nel dataset
# Puoi modificare 'max_folders' in base al numero di sotto-cartelle che vuoi processare
aus_data = process_all_images(dataset_path,0,200)

# Stampa i primi record per avere un'anteprima dei dati
print(aus_data.head())

Elaborazione della sotto-cartella: /content/dataset_resized/001
Errore durante l'elaborazione di /content/dataset_resized/001/00039.png: 
Errore durante l'elaborazione di /content/dataset_resized/001/00041.png: 
Errore durante l'elaborazione di /content/dataset_resized/001/00019.png: 
Errore durante l'elaborazione di /content/dataset_resized/001/00000.png: 
Errore durante l'elaborazione di /content/dataset_resized/001/00034.png: 
Errore durante l'elaborazione di /content/dataset_resized/001/00040.png: 
Errore durante l'elaborazione di /content/dataset_resized/001/00012.png: 
Errore durante l'elaborazione di /content/dataset_resized/001/00016.png: 
Errore durante l'elaborazione di /content/dataset_resized/001/00007.png: 
Errore durante l'elaborazione di /content/dataset_resized/001/00004.png: 
Errore durante l'elaborazione di /content/dataset_resized/001/00021.png: 
Errore durante l'elaborazione di /content/dataset_resized/001/00037.png: 
Errore durante l'elaborazione di /content/datase

<ipython-input-7-141764414a2d>:44: RuntimeWarning: divide by zero encountered in long_scalars
  sum_x = sum_x + land_x[i] * (1 / land_x[j])
<ipython-input-7-141764414a2d>:44: RuntimeWarning: invalid value encountered in double_scalars
  sum_x = sum_x + land_x[i] * (1 / land_x[j])


Errore durante l'elaborazione di /content/dataset_resized/021/00019.png: 


<ipython-input-7-141764414a2d>:44: RuntimeWarning: divide by zero encountered in long_scalars
  sum_x = sum_x + land_x[i] * (1 / land_x[j])
<ipython-input-7-141764414a2d>:44: RuntimeWarning: invalid value encountered in double_scalars
  sum_x = sum_x + land_x[i] * (1 / land_x[j])


Errore durante l'elaborazione di /content/dataset_resized/021/00036.png: 
Errore durante l'elaborazione di /content/dataset_resized/021/00016.png: 
Errore durante l'elaborazione di /content/dataset_resized/021/00007.png: 
Errore durante l'elaborazione di /content/dataset_resized/021/00037.png: 
Errore durante l'elaborazione di /content/dataset_resized/021/00002.png: 


...

Errore durante l'elaborazione di /content/dataset_resized/123/00010.png: 
Errore durante l'elaborazione di /content/dataset_resized/123/00014.png: 
Errore durante l'elaborazione di /content/dataset_resized/123/00009.png: 
Errore durante l'elaborazione di /content/dataset_resized/123/00008.png: 
Errore durante l'elaborazione di /content/dataset_resized/123/00011.png: 
Errore durante l'elaborazione di /content/dataset_resized/123/00026.png: 
Errore durante l'elaborazione di /content/dataset_resized/123/00025.png: 
Errore durante l'elaborazione di /content/dataset_resized/123/00051.png: 
Errore durante l'elaborazione di /content/dataset_resized/123/00055.png: 
Elaborazione della sotto-cartella: /content/dataset_resized/124
Elaborazione della sotto-cartella: /content/dataset_resized/125
Errore durante l'elaborazione di /content/dataset_resized/125/00012.png: 
Elaborazione della sotto-cartella: /content/dataset_resized/126


<ipython-input-7-141764414a2d>:44: RuntimeWarning: divide by zero encountered in long_scalars
  sum_x = sum_x + land_x[i] * (1 / land_x[j])
<ipython-input-7-141764414a2d>:44: RuntimeWarning: invalid value encountered in double_scalars
  sum_x = sum_x + land_x[i] * (1 / land_x[j])


Elaborazione della sotto-cartella: /content/dataset_resized/127
Elaborazione della sotto-cartella: /content/dataset_resized/128
Errore durante l'elaborazione di /content/dataset_resized/128/00034.png: 
Elaborazione della sotto-cartella: /content/dataset_resized/129
Elaborazione della sotto-cartella: /content/dataset_resized/130
Elaborazione della sotto-cartella: /content/dataset_resized/131
Errore durante l'elaborazione di /content/dataset_resized/131/00087.png: 
Errore durante l'elaborazione di /content/dataset_resized/131/00014.png: 
Elaborazione della sotto-cartella: /content/dataset_resized/132
Elaborazione della sotto-cartella: /content/dataset_resized/133
Errore durante l'elaborazione di /content/dataset_resized/133/00103.png: 
Errore durante l'elaborazione di /content/dataset_resized/133/00099.png: 
Errore durante l'elaborazione di /content/dataset_resized/133/00101.png: 
Errore durante l'elaborazione di /content/dataset_resized/133/00019.png: 
Errore durante l'elaborazione di /

<ipython-input-7-141764414a2d>:44: RuntimeWarning: divide by zero encountered in long_scalars
  sum_x = sum_x + land_x[i] * (1 / land_x[j])
<ipython-input-7-141764414a2d>:44: RuntimeWarning: invalid value encountered in double_scalars
  sum_x = sum_x + land_x[i] * (1 / land_x[j])
<ipython-input-7-141764414a2d>:44: RuntimeWarning: divide by zero encountered in long_scalars
  sum_x = sum_x + land_x[i] * (1 / land_x[j])
<ipython-input-7-141764414a2d>:44: RuntimeWarning: invalid value encountered in double_scalars
  sum_x = sum_x + land_x[i] * (1 / land_x[j])
<ipython-input-7-141764414a2d>:44: RuntimeWarning: divide by zero encountered in long_scalars
  sum_x = sum_x + land_x[i] * (1 / land_x[j])
<ipython-input-7-141764414a2d>:44: RuntimeWarning: invalid value encountered in double_scalars
  sum_x = sum_x + land_x[i] * (1 / land_x[j])
<ipython-input-7-141764414a2d>:44: RuntimeWarning: divide by zero encountered in long_scalars
  sum_x = sum_x + land_x[i] * (1 / land_x[j])
<ipython-input-7-

Elaborazione della sotto-cartella: /content/dataset_resized/154
Elaborazione della sotto-cartella: /content/dataset_resized/155
Elaborazione della sotto-cartella: /content/dataset_resized/156
Errore durante l'elaborazione di /content/dataset_resized/156/00117.png: 
Errore durante l'elaborazione di /content/dataset_resized/156/00111.png: 
Errore durante l'elaborazione di /content/dataset_resized/156/00115.png: 
Errore durante l'elaborazione di /content/dataset_resized/156/00116.png: 
Errore durante l'elaborazione di /content/dataset_resized/156/00112.png: 
Elaborazione della sotto-cartella: /content/dataset_resized/157
Elaborazione della sotto-cartella: /content/dataset_resized/158
Elaborazione della sotto-cartella: /content/dataset_resized/159
Elaborazione della sotto-cartella: /content/dataset_resized/160
Elaborazione della sotto-cartella: /content/dataset_resized/161
Elaborazione della sotto-cartella: /content/dataset_resized/162
Elaborazione della sotto-cartella: /content/dataset_re

In [ ]:
len(aus_data)

9181

In [ ]:
aus_data.to_csv('afew_0_200.csv')